In [1]:
import sys
import numpy as np
import import_ipynb
from scipy.sparse.linalg import spsolve
from pathlib import Path

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from fem_pre_processing import read_mesh
    from fem_processing import boundary_conditions, matrices_assembly
    from fem_pos_processing import graph_results
    print("Modules imports were successful!")
except ModuleNotFoundError as en:
    print(f"Modules were not found: {en}")
except ImportError as en:
    print(f"Error in import: {en}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!


# Project 1: Poisson Problem at Rectangular Domain $\Omega = [0,1]^2$

Considere um problema de _Poisson_ 2D em $\Omega = [0,1]^2$

$$
-\nabla \cdot \left( \nabla u(x,y) \right) = f(x,y)
\tag{1}
$$

onde,
$$
f(x,y) = 2\pi^2 \, sin(\pi x) \, sin(\pi y)
\tag{2}
$$

cujas condições de contorno são $u=0$ em $\partial \Omega$.

A solução analítica deste problema é

$$
u(x,y) = sin(\pi x) \, sin(\pi y)
\tag{3}
$$

Implemente um programa de elementos finitos utilizando elementos triangulares $P_1$ para resolver o Problema de Valor de Contorno $(1)$.

# 6-noded Quadratic Triangular Elements, $P_2$
## Pre-processor module
## Geometry and mesh Domain

In [2]:
FINITE_ELEMENT = ("Triangle", 2)
BOUNDARY = [{'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'omega'}]
MATERIAL = [{'tag': 201, 'name': 'free_space', 'a_constant': 1}]

# Create mesh from file geometry
ps.create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=1E-1, view_mesh=False)

# Read mesh data
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='rectangular', info_mode=False)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pos_processing\pictures\analytical_solution.svg
Model rectangular_domain_Triangle2 (2D)
Info     : 9 geometric entities
Info     : 2 Physical Groups
Info     : 525 nodes in total
Info     : 242 2-D elements in total


## Mesh Data Dictionaries

In [3]:
cell_data = mesh_data['cell']; cell_data

{1: {'Tag': 45,
  'conn': [112, 121, 142, 183, 184, 185],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 2: {'Tag': 46,
  'conn': [162, 116, 164, 186, 187, 188],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 3: {'Tag': 47,
  'conn': [155, 89, 160, 189, 190, 191],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 4: {'Tag': 48,
  'conn': [146, 92, 161, 192, 193, 194],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 5: {'Tag': 49,
  'conn': [92, 146, 162, 192, 195, 196],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 6: {'Tag': 50,
  'conn': [89, 155, 174, 189, 197, 198],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 7: {'Tag': 51,
  'conn': [136, 162, 164, 199, 188, 200],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 8: {'Tag': 52,
  'conn': [87, 126, 147, 201, 202, 203],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}

In [4]:
nodes_data = mesh_data['nodes']; nodes_data

{1: {'xg': (0.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 2: {'xg': (1.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 3: {'xg': (1.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 4: {'xg': (0.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 5: {'xg': (0.09999999999981467, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 6: {'xg': (0.1999999999995579, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 7: {'xg': (0.2999999999992664, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 8: {'xg': (0.3999999999989749, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 9: {'xg': (0.4999999999986943, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 10: {'xg': (0.5999999999989468, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 11: {'xg': (0.69999999999921, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 12: {'xg': (0.7999999999994734, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 13: {'xg': (0.8999999999997368, 0.0),
  'bc': {'type': 'D

In [5]:
conn = {key: cell['conn'] for key, cell in cell_data.items()}; conn

{1: [112, 121, 142, 183, 184, 185],
 2: [162, 116, 164, 186, 187, 188],
 3: [155, 89, 160, 189, 190, 191],
 4: [146, 92, 161, 192, 193, 194],
 5: [92, 146, 162, 192, 195, 196],
 6: [89, 155, 174, 189, 197, 198],
 7: [136, 162, 164, 199, 188, 200],
 8: [87, 126, 147, 201, 202, 203],
 9: [92, 162, 165, 196, 204, 205],
 10: [133, 82, 135, 206, 207, 208],
 11: [129, 93, 131, 209, 210, 211],
 12: [162, 136, 165, 199, 212, 204],
 13: [142, 121, 167, 184, 213, 214],
 14: [147, 126, 166, 202, 215, 216],
 15: [119, 112, 142, 217, 185, 218],
 16: [82, 133, 175, 206, 219, 220],
 17: [131, 93, 169, 210, 221, 222],
 18: [131, 86, 132, 223, 224, 225],
 19: [132, 86, 133, 224, 226, 227],
 20: [126, 87, 167, 201, 228, 229],
 21: [135, 82, 136, 207, 230, 231],
 22: [94, 155, 160, 232, 191, 233],
 23: [95, 146, 161, 234, 194, 235],
 24: [123, 139, 174, 236, 237, 238],
 25: [93, 129, 159, 209, 239, 240],
 26: [163, 96, 166, 241, 242, 243],
 27: [126, 127, 166, 244, 245, 215],
 28: [168, 88, 173, 246, 247

## Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$

In [6]:
graph_results.plot_mesh(FINITE_ELEMENT, mesh_data, model='rectangular', Numbering=True)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pre_processing\pictures\rectangular_meshed_domain_Triangle2.svg


<figure>
    <img src="pre_processing/pictures/rectangular_meshed_domain_Triangle2.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$.</figcaption>
</figure>

## `apply_physics()`

In [7]:
mesh_data = ps.apply_physics(FINITE_ELEMENT, mesh_data); mesh_data['cell']

{1: {'Tag': 45,
  'conn': [112, 121, 142, 183, 184, 185],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1},
  'stiffness_a_value': 1,
  'mass_a_value': 1,
  'source': {'type': 'analytical_source_fx', 'value': 13.469577046948057}},
 2: {'Tag': 46,
  'conn': [162, 116, 164, 186, 187, 188],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1},
  'stiffness_a_value': 1,
  'mass_a_value': 1,
  'source': {'type': 'analytical_source_fx', 'value': 11.878527416397793}},
 3: {'Tag': 47,
  'conn': [155, 89, 160, 189, 190, 191],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1},
  'stiffness_a_value': 1,
  'mass_a_value': 1,
  'source': {'type': 'analytical_source_fx', 'value': 4.074438078888525}},
 4: {'Tag': 48,
  'conn': [146, 92, 161, 192, 193, 194],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1},
  'stiffness_a_value': 1,
  'mass_a_value': 1,
  'source': {'type': 'analytical_source_fx', 'value': 4.090043563149103}},
 5: {'Tag': 

## ``global_nodes_coordinates()``

In [8]:
nodes_coord = {key: value['xg'] for key, value in nodes_data.items()}; nodes_coord

{1: (0.0, 0.0),
 2: (1.0, 0.0),
 3: (1.0, 1.0),
 4: (0.0, 1.0),
 5: (0.09999999999981467, 0.0),
 6: (0.1999999999995579, 0.0),
 7: (0.2999999999992664, 0.0),
 8: (0.3999999999989749, 0.0),
 9: (0.4999999999986943, 0.0),
 10: (0.5999999999989468, 0.0),
 11: (0.69999999999921, 0.0),
 12: (0.7999999999994734, 0.0),
 13: (0.8999999999997368, 0.0),
 14: (0.04999999999989927, 0.0),
 15: (0.1499999999997036, 0.0),
 16: (0.2499999999994121, 0.0),
 17: (0.3499999999991207, 0.0),
 18: (0.4499999999988346, 0.0),
 19: (0.5499999999988205, 0.0),
 20: (0.6499999999990784, 0.0),
 21: (0.7499999999993416, 0.0),
 22: (0.8499999999996051, 0.0),
 23: (0.9499999999998684, 0.0),
 24: (1.0, 0.09999999999981467),
 25: (1.0, 0.1999999999995579),
 26: (1.0, 0.2999999999992664),
 27: (1.0, 0.3999999999989749),
 28: (1.0, 0.4999999999986943),
 29: (1.0, 0.5999999999989468),
 30: (1.0, 0.69999999999921),
 31: (1.0, 0.7999999999994734),
 32: (1.0, 0.8999999999997368),
 33: (1.0, 0.04999999999989927),
 34: (1.0, 0.

## $x_g, y_g$ global coordinate

In [9]:
xg = {key: value['xg'][0] for key, value in nodes_data.items()}; xg

{1: 0.0,
 2: 1.0,
 3: 1.0,
 4: 0.0,
 5: 0.09999999999981467,
 6: 0.1999999999995579,
 7: 0.2999999999992664,
 8: 0.3999999999989749,
 9: 0.4999999999986943,
 10: 0.5999999999989468,
 11: 0.69999999999921,
 12: 0.7999999999994734,
 13: 0.8999999999997368,
 14: 0.04999999999989927,
 15: 0.1499999999997036,
 16: 0.2499999999994121,
 17: 0.3499999999991207,
 18: 0.4499999999988346,
 19: 0.5499999999988205,
 20: 0.6499999999990784,
 21: 0.7499999999993416,
 22: 0.8499999999996051,
 23: 0.9499999999998684,
 24: 1.0,
 25: 1.0,
 26: 1.0,
 27: 1.0,
 28: 1.0,
 29: 1.0,
 30: 1.0,
 31: 1.0,
 32: 1.0,
 33: 1.0,
 34: 1.0,
 35: 1.0,
 36: 1.0,
 37: 1.0,
 38: 1.0,
 39: 1.0,
 40: 1.0,
 41: 1.0,
 42: 1.0,
 43: 0.8999999999995836,
 44: 0.7999999999999998,
 45: 0.7000000000006934,
 46: 0.6000000000013869,
 47: 0.5000000000020587,
 48: 0.4000000000016644,
 49: 0.3000000000012483,
 50: 0.2000000000008322,
 51: 0.100000000000416,
 52: 0.9499999999998356,
 53: 0.8499999999998515,
 54: 0.7500000000003858,
 55: 

## $a_e = (x_e, y_e)$ global element coordinate

In [10]:
ai = {key: [nodes_data[id]['xg'] for id in node_ids] for key, node_ids in conn.items()}
xi = {key: [coord[0] for coord in coords] for key, coords in ai.items()}
yi = {key: [coord[1] for coord in coords] for key, coords in ai.items()}
print("'a_e' global coordinates: "); ai

'a_e' global coordinates: 


{1: [(0.7044542185661746, 0.4836609274158983),
  (0.753535810939751, 0.3987254670893935),
  (0.8167956118738326, 0.4899817334731329),
  (0.7289950147529628, 0.4411931972526459),
  (0.7851657114067918, 0.4443536002812632),
  (0.7606249152200036, 0.4868213304445156)],
 2: [(0.1890575670468492, 0.2940842362157938),
  (0.300000000002935, 0.3071796769753505),
  (0.2503952377171335, 0.3884573638917699),
  (0.2445287835248921, 0.3006319565955722),
  (0.2751976188600342, 0.3478185204335602),
  (0.2197264023819913, 0.3412708000537819)],
 3: [(0.815040741820152, 0.8183024367054095),
  (0.9103703875043379, 0.7586725654707323),
  (0.9167537064254059, 0.8508718537559959),
  (0.8627055646622449, 0.7884875010880709),
  (0.9135620469648719, 0.804772209613364),
  (0.8658972241227789, 0.8345871452307028)],
 4: [(0.1856343100128922, 0.1794039491425674),
  (0.0866025403788042, 0.2500000000010402),
  (0.08296629115975593, 0.1498875452108657),
  (0.1361184251958482, 0.2147019745718038),
  (0.084784415769280

In [11]:
print("'a_1' global coordinates: "); ai[1]

'a_1' global coordinates: 


[(0.7044542185661746, 0.4836609274158983),
 (0.753535810939751, 0.3987254670893935),
 (0.8167956118738326, 0.4899817334731329),
 (0.7289950147529628, 0.4411931972526459),
 (0.7851657114067918, 0.4443536002812632),
 (0.7606249152200036, 0.4868213304445156)]

In [12]:
print("'Node 2' global coordinates: "); nodes_data[2]['xg']

'Node 2' global coordinates: 


(1.0, 0.0)

In [13]:
print("'x_1' x-coordinates: "); xi[1]

'x_1' x-coordinates: 


[0.7044542185661746,
 0.753535810939751,
 0.8167956118738326,
 0.7289950147529628,
 0.7851657114067918,
 0.7606249152200036]

## `map_to_physical_coordinates()`

In [14]:
xi_master = (0, 1)
xg_1, yg_1 = matrices_assembly.isomapping_to_global_coordinates(ai[1], xi_master, FINITE_ELEMENT)
print(f"'e_1' Master coordinates: {xi_master} --> Global coordinates: ({xg_1}, {yg_1})")

'e_1' Master coordinates: (0, 1) --> Global coordinates: (0.8167956118738326, 0.4899817334731329)


## Material Proprieties, $k_a$

In [15]:
ka = {key: value['stiffness_a_value'] for key, value in cell_data.items()}; ka

{1: 1,
 2: 1,
 3: 1,
 4: 1,
 5: 1,
 6: 1,
 7: 1,
 8: 1,
 9: 1,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 1,
 18: 1,
 19: 1,
 20: 1,
 21: 1,
 22: 1,
 23: 1,
 24: 1,
 25: 1,
 26: 1,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 1,
 32: 1,
 33: 1,
 34: 1,
 35: 1,
 36: 1,
 37: 1,
 38: 1,
 39: 1,
 40: 1,
 41: 1,
 42: 1,
 43: 1,
 44: 1,
 45: 1,
 46: 1,
 47: 1,
 48: 1,
 49: 1,
 50: 1,
 51: 1,
 52: 1,
 53: 1,
 54: 1,
 55: 1,
 56: 1,
 57: 1,
 58: 1,
 59: 1,
 60: 1,
 61: 1,
 62: 1,
 63: 1,
 64: 1,
 65: 1,
 66: 1,
 67: 1,
 68: 1,
 69: 1,
 70: 1,
 71: 1,
 72: 1,
 73: 1,
 74: 1,
 75: 1,
 76: 1,
 77: 1,
 78: 1,
 79: 1,
 80: 1,
 81: 1,
 82: 1,
 83: 1,
 84: 1,
 85: 1,
 86: 1,
 87: 1,
 88: 1,
 89: 1,
 90: 1,
 91: 1,
 92: 1,
 93: 1,
 94: 1,
 95: 1,
 96: 1,
 97: 1,
 98: 1,
 99: 1,
 100: 1,
 101: 1,
 102: 1,
 103: 1,
 104: 1,
 105: 1,
 106: 1,
 107: 1,
 108: 1,
 109: 1,
 110: 1,
 111: 1,
 112: 1,
 113: 1,
 114: 1,
 115: 1,
 116: 1,
 117: 1,
 118: 1,
 119: 1,
 120: 1,
 121: 1,
 122: 1,
 123: 1,
 

## Source: Analytical source, $f(x)$

In [16]:
fx = {key: value['source']['value'] for key, value in cell_data.items()}; fx

{1: 13.469577046948057,
 2: 11.878527416397793,
 3: 4.074438078888525,
 4: 4.090043563149103,
 5: 6.300848623109182,
 6: 5.8484702552512715,
 7: 10.444337426311748,
 8: 5.940916468246733,
 9: 5.777795505112335,
 10: 8.785596384989951,
 11: 5.051001166628853,
 12: 7.57345588401181,
 13: 11.137044036433647,
 14: 6.212614671281762,
 15: 13.605287062811296,
 16: 6.183749717838805,
 17: 4.006493475031684,
 18: 7.296508501485978,
 19: 8.02775316897733,
 20: 7.752942392645781,
 21: 8.900648706547628,
 22: 3.5318192693096386,
 23: 3.5227128037020936,
 24: 7.86220618821271,
 25: 3.6906052292356604,
 26: 3.610085525773199,
 27: 8.023419587299903,
 28: 7.107175776802518,
 29: 18.619169104039422,
 30: 12.796773243824529,
 31: 8.462813368908067,
 32: 13.891772408997452,
 33: 5.650693230460518,
 34: 15.185559515879174,
 35: 17.908789409765387,
 36: 17.55166849983177,
 37: 10.959923735580992,
 38: 4.7575671019228105,
 39: 4.839683286471004,
 40: 13.769075290992273,
 41: 9.473660627502491,
 42: 10.733

## Jacobian Matrix Transform

In [17]:
Je = matrices_assembly.jacobian(FINITE_ELEMENT, mesh_data, e=1, xik=(0, 0)) 
Jdet, Jinv = np.linalg.det(Je), np.linalg.inv(Je)
print("Jacobian matrix for element e_1:\n", Je)
print("Determinant of the Jacobian matrix for element e_1:", Jdet)

Jacobian matrix for element e_1:
 [[ 0.04908159 -0.08493546]
 [ 0.11234139  0.00632081]]
Determinant of the Jacobian matrix for element e_1: 0.00985200318068048


## Local Elements $e_1$

In [18]:
Ae, fe, Me, area_e = matrices_assembly.local_matrices(FINITE_ELEMENT, mesh_data, e=1)
print("Local stiffness matrix for element e_1:\n", Ae)
print("Local load vector for element e_1:\n", fe)
print("Area of element e_1: ", area_e)

Local stiffness matrix for element e_1:
 [[ 6.25736122e-01+0.j  1.29982149e-01+0.j  7.85965571e-02+0.j
  -5.19928596e-01+0.j -4.11154888e-09+0.j -3.14386227e-01+0.j]
 [ 1.29982149e-01+0.j  6.42536390e-01+0.j  8.41966486e-02+0.j
  -5.19928596e-01+0.j -3.36786594e-01+0.j  3.32911776e-09+0.j]
 [ 7.85965571e-02+0.j  8.41966486e-02+0.j  4.88379615e-01+0.j
   2.33294802e-09+0.j -3.36786593e-01+0.j -3.14386229e-01+0.j]
 [-5.19928596e-01+0.j -5.19928596e-01+0.j  2.33294802e-09+0.j
   2.34220284e+00+0.j -6.28772450e-01+0.j -6.73573195e-01+0.j]
 [-4.11154888e-09+0.j -3.36786594e-01+0.j -3.36786593e-01+0.j
  -6.28772450e-01+0.j  2.34220283e+00+0.j -1.03985718e+00+0.j]
 [-3.14386227e-01+0.j  3.32911776e-09+0.j -3.14386229e-01+0.j
  -6.73573195e-01+0.j -1.03985718e+00+0.j  2.34220283e+00+0.j]]
Local load vector for element e_1:
 [[ 5.31448728e-11+0.j]
 [-9.01415227e-13+0.j]
 [-9.01423033e-13+0.j]
 [ 2.21170525e-02+0.j]
 [ 2.21170523e-02+0.j]
 [ 2.21170525e-02+0.j]]
Area of element e_1:  0.004926001

## Global “stiffness” matrix, $A_g$

In [19]:
Ag, fg, Mg = matrices_assembly.global_matrix(FINITE_ELEMENT, mesh_data)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (525, 525)


## Imposition of Boundary Conditions
### $Dirichlet$ nodes

In [20]:
dirichlet_nodes = {key: value 
                   for key, value in nodes_data.items() if value['bc']['type'] == 'Dirichlet'}
dirichlet_nodes

{1: {'xg': (0.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 2: {'xg': (1.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 3: {'xg': (1.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 4: {'xg': (0.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 5: {'xg': (0.09999999999981467, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 6: {'xg': (0.1999999999995579, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 7: {'xg': (0.2999999999992664, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 8: {'xg': (0.3999999999989749, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 9: {'xg': (0.4999999999986943, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 10: {'xg': (0.5999999999989468, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 11: {'xg': (0.69999999999921, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 12: {'xg': (0.7999999999994734, 0.0),
  'bc': {'type': 'Dirichlet', 'value': 0.0}},
 13: {'xg': (0.8999999999997368, 0.0),
  'bc': {'type': 'D

In [21]:
Nn = len(nodes_data); Nd = len(dirichlet_nodes)
print(f"The entire domain has {Nn} nodes: {Nn - Nd} free nodes; {Nd} Dirichlet nodes.")

The entire domain has 525 nodes: 445 free nodes; 80 Dirichlet nodes.


In [22]:
free_nodes = {key: value for key, value in nodes_data.items() if value['bc']['type'] != 'Dirichlet'}; free_nodes

{81: {'xg': (0.4500000000018615, 0.9133974596216763),
  'bc': {'type': 'Free', 'value': None}},
 82: {'xg': (0.0923679274106518, 0.4605742807381246),
  'bc': {'type': 'Free', 'value': None}},
 83: {'xg': (0.5499999999988205, 0.08660254037843171),
  'bc': {'type': 'Free', 'value': None}},
 84: {'xg': (0.9152217186777625, 0.546840290627498),
  'bc': {'type': 'Free', 'value': None}},
 85: {'xg': (0.6500000000008002, 0.9133974596219514),
  'bc': {'type': 'Free', 'value': None}},
 86: {'xg': (0.08885430207258675, 0.6509201999022066),
  'bc': {'type': 'Free', 'value': None}},
 87: {'xg': (0.9133974596218086, 0.3499999999991206),
  'bc': {'type': 'Free', 'value': None}},
 88: {'xg': (0.3475303635242558, 0.08190748804974743),
  'bc': {'type': 'Free', 'value': None}},
 89: {'xg': (0.9103703875043379, 0.7586725654707323),
  'bc': {'type': 'Free', 'value': None}},
 90: {'xg': (0.250817301762687, 0.9138693290130382),
  'bc': {'type': 'Free', 'value': None}},
 91: {'xg': (0.7459788032314767, 0.0795

## Mapping Global nodes to reduced system

This code creates a dictionary called ``global_to_reduced``, which maps the global indices of the mesh nodes to the reduced indices, i.e. the indices that correspond only to the nodes that are not in the _Dirichlet boundary conditions_.

In [23]:
{global_id: idx + 1 for idx, global_id in enumerate(free_nodes.keys())}

{81: 1,
 82: 2,
 83: 3,
 84: 4,
 85: 5,
 86: 6,
 87: 7,
 88: 8,
 89: 9,
 90: 10,
 91: 11,
 92: 12,
 93: 13,
 94: 14,
 95: 15,
 96: 16,
 97: 17,
 98: 18,
 99: 19,
 100: 20,
 101: 21,
 102: 22,
 103: 23,
 104: 24,
 105: 25,
 106: 26,
 107: 27,
 108: 28,
 109: 29,
 110: 30,
 111: 31,
 112: 32,
 113: 33,
 114: 34,
 115: 35,
 116: 36,
 117: 37,
 118: 38,
 119: 39,
 120: 40,
 121: 41,
 122: 42,
 123: 43,
 124: 44,
 125: 45,
 126: 46,
 127: 47,
 128: 48,
 129: 49,
 130: 50,
 131: 51,
 132: 52,
 133: 53,
 134: 54,
 135: 55,
 136: 56,
 137: 57,
 138: 58,
 139: 59,
 140: 60,
 141: 61,
 142: 62,
 143: 63,
 144: 64,
 145: 65,
 146: 66,
 147: 67,
 148: 68,
 149: 69,
 150: 70,
 151: 71,
 152: 72,
 153: 73,
 154: 74,
 155: 75,
 156: 76,
 157: 77,
 158: 78,
 159: 79,
 160: 80,
 161: 81,
 162: 82,
 163: 83,
 164: 84,
 165: 85,
 166: 86,
 167: 87,
 168: 88,
 169: 89,
 170: 90,
 171: 91,
 172: 92,
 173: 93,
 174: 94,
 175: 95,
 176: 96,
 177: 97,
 178: 98,
 179: 99,
 180: 100,
 181: 101,
 182: 102,
 183:

## Processor Module
## Asymmetric Global matrix $A_g$ with boundary conditions

In [24]:
Ag, fg = boundary_conditions.apply_simple_dirichlet(Ag, fg, mesh_data)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (525, 525)


## Global Potential Vector

In [25]:
# Make sure Ag is in CSR format for efficiency
# Convert fg to a dense format or keep it sparse as needed
# Solve the linear system Ag * u = fg
ug = spsolve(Ag.tocsr(), fg.toarray())
print("The solution vector u is: \n", ug)

The solution vector u is: 
 [0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +

## Reduced Global matrix $A_{gr}$ with boundary conditions

In [26]:
Agr, fgr, Mgr = boundary_conditions.reduced_global_matrices(FINITE_ELEMENT, mesh_data)
print("Reduced global matrix shape:", Agr.shape)

Reduced global matrix shape: (445, 445)


## Global Potential Solution

In [27]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.global_potentials_solution(mesh_data, ur)
print("Reduced solution vector ur: ", ur)
print("Global potential vector V: "); potential_u

Reduced solution vector ur:  [0.26433557+0.j 0.28300495+0.j 0.26408185+0.j 0.25937532+0.j
 0.23845348+0.j 0.24432497+0.j 0.23849375+0.j 0.22492748+0.j
 0.19015583+0.j 0.18874142+0.j 0.17637313+0.j 0.18948934+0.j
 0.13422341+0.j 0.11919332+0.j 0.1192086 +0.j 0.11695713+0.j
 0.23830637+0.j 0.49032328+0.j 0.51557001+0.j 0.71666734+0.j
 0.71666847+0.j 0.8822601 +0.j 0.83828703+0.j 0.96142509+0.j
 0.96204234+0.j 0.9941772 +0.j 0.94551749+0.j 0.94483417+0.j
 0.92964518+0.j 0.84027809+0.j 0.86789248+0.j 0.79647035+0.j
 0.8380687 +0.j 0.80245536+0.j 0.77929198+0.j 0.66263184+0.j
 0.81912852+0.j 0.62889316+0.j 0.68389741+0.j 0.71278671+0.j
 0.66152316+0.j 0.66252425+0.j 0.51868997+0.j 0.52007832+0.j
 0.62888688+0.j 0.45565491+0.j 0.45361411+0.j 0.41706885+0.j
 0.29775119+0.j 0.51167767+0.j 0.32563418+0.j 0.51140555+0.j
 0.43762452+0.j 0.68143594+0.j 0.5850792 +0.j 0.45411359+0.j
 0.86615758+0.j 0.49033501+0.j 0.27423923+0.j 0.23823836+0.j
 0.25726197+0.j 0.54188353+0.j 0.25621893+0.j 0.26433151

{1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: (0.2643355729861439+0j),
 82: (0.28300495244408985+0j),
 83: (0.26408184609114926+0j),
 84: (0.2593753161474838+0j),
 85: (0.23845348378127354+0j),
 86: (0.24432497251180535+0j),
 87: (0.238493750025004+0

## Solution at each element

In [28]:
u_for_cell = {}
for en, cell in mesh_data['cell'].items():
    u_e = []
    for node in cell['conn']:
        u_e.append(potential_u[node])
    u_for_cell[en] = u_e

print("The complete solution for each cell."); u_for_cell

The complete solution for each cell.


{1: [(0.7964703501384774+0j),
  (0.6615231563712757+0j),
  (0.5418835252943072+0j),
  (0.7365285936333241+0j),
  (0.6130766466938933+0j),
  (0.6800331921751321+0j)],
 2: [(0.44479794505811365+0j),
  (0.6626318435186598+0j),
  (0.6624719819419316+0j),
  (0.5610709945698914+0j),
  (0.6730264788337443+0j),
  (0.5573009767605713+0j)],
 3: [(0.2952772279709174+0j),
  (0.19015583152888915+0j),
  (0.11628296214012988+0j),
  (0.2566810276150599+0j),
  (0.15375657375291896+0j),
  (0.2021296737364472+0j)],
 4: [(0.29301345080729907+0j),
  (0.18948933543266525+0j),
  (0.11655572632124668+0j),
  (0.25814007599561367+0j),
  (0.1542641570059235+0j),
  (0.20171122851530188+0j)],
 5: [(0.18948933543266525+0j),
  (0.29301345080729907+0j),
  (0.44479794505811365+0j),
  (0.25814007599561367+0j),
  (0.37438082612648066+0j),
  (0.31531544470090017+0j)],
 6: [(0.19015583152888915+0j),
  (0.2952772279709174+0j),
  (0.37485641369764916+0j),
  (0.2566810276150599+0j),
  (0.3410101716114168+0j),
  (0.2814905716

## Interpolate Solution at element, $e_n$

In [29]:
xi_master = (0, 0.5)
en = 1
u_at_node = matrices_assembly.isomapping_to_global_coordinates(u_for_cell[en], xi_master, FINITE_ELEMENT)

print(f"Element e_{en}: Potential 'u' interpolate at xi = {xi_master} --> u(xi): {u_at_node}")

Element e_1: Potential 'u' interpolate at xi = (0, 0.5) --> u(xi): [0.68003319+0.j]


## Post-Processor

In [30]:
graph_results.fem_solution(FINITE_ELEMENT, mesh_data, potential_u, type='real')
graph_results.fem_griddata(FINITE_ELEMENT, mesh_data, potential_u, type='real')

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pos_processing\pictures\fem_solution_Triangle2.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pos_processing\pictures\fem_solution_griddata_Triangle2.svg


### Figure 2: Scalar Field Distribution on a Triangular Mesh
<figure>
    <img src="pos_processing/pictures/fem_solution_Triangle2.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Post-Processor: Scalar Field Distribution.</figcaption>
</figure>

### Figure 3: Scalar Field Distribution
<figure>
    <img src="pos_processing/pictures/fem_solution_griddata_Triangle2.svg" alt="Fig.3" style="width:100%;" />
    <figcaption>Figure 3: Post-Processor: Scalar Field Distribution and Triangular Mesh.</figcaption>
</figure>

### Figure 4: Analytical Scalar Field Distribution on a Triangular Mesh
<figure>
    <img src="pos_processing/pictures/analytical_solution.svg" alt="Fig.4" style="width:100%;" />
    <figcaption>Figure 4: Analytical Scalar Field Distribution on a Triangular Mesh.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.